# Parkinson's Mouse Tutorial - Taxonomy Assignment

Run this notebook in `qiime2-2022.11`.

Continuing the [pd-mouse tutorial](https://docs.qiime2.org/2022.11/tutorials/pd-mice/). Specifically the [Taxonomy](https://docs.qiime2.org/2022.11/tutorials/pd-mice/#taxonomic-classification), and [Phylogeny](https://docs.qiime2.org/2022.11/tutorials/pd-mice/#generating-a-phylogenetic-tree-for-diversity-analysis) steps. *Note we'll use a *de novo* [align-to-tree-mafft-fasttree ](https://docs.qiime2.org/2022.11/tutorials/phylogeny/#pipelines) step so we can run through this tutorial quicker.*

In [12]:
from os import getcwd, listdir, chdir, mkdir
import qiime2 as q2

In [13]:
getcwd()

'/home/fmshaik/BMIG-62033/processed'

In [15]:
chdir('/home/fmshaik/BMIG-62033/processed')
getcwd()

'/home/fmshaik/BMIG-62033/processed'

## Download classifiers if runing on your laptop:

We'll assign taxonomy using SILVA. Can obtain classifiers from the [Data Resource Page](https://docs.qiime2.org/2022.11/data-resources/).

## If you are running on the HPC the classifiers are located at:
 - `/home/SE/BMIG-6202-MSR/RefDBs/q2-2022.11/silva-138-1-ssu-nr99-515f-806r-classifier.qza`
 - `/home/SE/BMIG-6202-MSR/RefDBs/q2-2022.11/silva-138-1-ssu-nr99-classifier.qza`
 
 You can setup a shortcut like this:

V4:
`silva_classifier='/home/SE/BMIG-6202-MSR/RefDBs/q2-2023.9/silva-138-1-ssu-nr99-515f-806r-classifier.qza'`

V3V4:
`silva_classifier='/home/SE/BMIG-6202-MSR/RefDBs/q2-2023.9/silva-138-1-ssu-nr99-357f-785r-classifier.qza'`

In [ ]:
silva_classifier='/home/SE/BMIG-6202-MSR/RefDBs/q2-2023.9/with-species/silva-138-1-ssu-nr99-sl-515f-806r-classifier.qza'

## Classify sequences / reads

In the command below, I'll be running on the HPC using the shortcut `$silva_classifier`.

In [16]:
! qiime feature-classifier classify-sklearn \
    --i-reads ./dada2_rep_set.qza \
    --i-classifier $silva_classifier \
    --p-n-jobs 2 \
    --o-classification ./taxonomy.qza

Saved FeatureData[Taxonomy] to: ./taxonomy.qza


In [17]:
# View list of classifications
! qiime metadata tabulate \
    --m-input-file ./taxonomy.qza \
    --o-visualization ./taxonomy.qzv

Saved Visualization to: ./taxonomy.qzv


In [18]:
# View a taxonomy barplot
! qiime taxa barplot \
    --i-table ./dada2_table.qza \
    --i-taxonomy ./taxonomy.qza \
    --m-metadata-file ./metadata.tsv \
    --o-visualization ./taxa_barplot.qzv

Saved Visualization to: ./taxa_barplot.qzv


## Remove poorly classified reads

[Filtering Documentation](https://docs.qiime2.org/2020.11/tutorials/filtering/)

In [19]:
! qiime taxa filter-table \
    --i-table ./dada2_table.qza \
    --i-taxonomy ./taxonomy.qza \
    --p-mode 'contains'  \
    --p-include 'p__' \
    --p-exclude 'p__;,Eukaryota,Chloroplast,Mitochondria' \
    --o-filtered-table ./table-no-ecmu.qza

Saved FeatureTable[Frequency] to: ./table-no-ecmu.qza


In [ ]:
! qiime metadata tabulate \
    --m-input-file ./taxonomy.qza \
    --o-visualization ./taxonomy.qzv

In [23]:
# summarize ESV table
! qiime feature-table summarize \
    --i-table ./table-no-ecmu.qza \
    --m-sample-metadata-file ./metadata.tsv \
    --o-visualization ./table-no-ecmu.qzv

Saved Visualization to: ./table-no-ecmu.qzv


In [27]:
 #keep seq file in sync with table! 
! qiime feature-table filter-seqs \
    --i-data ./dada2_rep_set.qza \
    --i-table ./table-no-ecmu.qza \
    --o-filtered-data rep_set-no-ecmu.qza

Saved FeatureData[Sequence] to: rep_set-no-ecmu.qza


In [28]:
! qiime tools export \
    --input-path rep_set-no-ecmu.qza \
    --output-path rep_set-no-ecmu-export

Exported rep_set-no-ecmu.qza as DNASequencesDirectoryFormat to directory rep_set-no-ecmu-export


In [29]:
# View a taxonomy barplot
! qiime taxa barplot \
    --i-table ./table-no-ecmu.qza \
    --i-taxonomy ./taxonomy.qza \
    --m-metadata-file ./metadata.tsv \
    --o-visualization ./table-no-ecmu-taxa-barplot.qzv

Saved Visualization to: ./table-no-ecmu-taxa-barplot.qzv


#### krona plot

In [25]:
! qiime krona collapse-and-plot \
    --i-table ./table-no-ecmu.qza \
    --i-taxonomy ./taxonomy.qza \
    --o-krona-plot ./table-no-ecmu-taxa-krona.qzv

Saved Visualization to: ./table-no-ecmu-taxa-krona.qzv


##  Other QA / QC Operations

See [q2-quality-control tutorial](https://docs.qiime2.org/2022.11/tutorials/quality-control/).

In [30]:
silva_ref_seq='/home/SE/BMIG-6202-MSR/RefDBs/q2-2023.9/with-species/silva-138-1-ssu-nr99-sl-seqs-515f-806r-cln-dr-uniq.qza'

In [31]:
# remove poor quality sequence that do not have a decent match to our curated reference database.
! qiime quality-control exclude-seqs \
    --i-query-sequences ./rep_set-no-ecmu.qza \
    --i-reference-sequences $silva_ref_seq \
    --p-method vsearch \
    --p-perc-identity 0.90 \
    --p-perc-query-aligned 0.90 \
    --p-threads 8 \
    --o-sequence-hits ./hits.qza \
    --o-sequence-misses ./misses.qza \
    --verbose

Running external command line application. This may print messages to stdout and/or stderr.
The commands to be run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: vsearch --usearch_global /tmp/qiime2/fmshaik/data/b332decb-d1ec-4ec6-8065-a66dc437893b/data/dna-sequences.fasta --id 0.9 --strand both --maxaccepts 1 --maxrejects 0 --db /tmp/qiime2/fmshaik/data/83d90385-3fcf-4bf8-99a4-b570ef37d89e/data/dna-sequences.fasta --threads 8 --userfields query+target+ql+qlo+qhi --userout /tmp/tmpxtt2orno

vsearch v2.22.1_linux_x86_64, 125.6GB RAM, 56 cores
https://github.com/torognes/vsearch

Reading file /tmp/qiime2/fmshaik/data/83d90385-3fcf-4bf8-99a4-b570ef37d89e/data/dna-sequences.fasta 100%          
94927040 nt in 338557 seqs, min 151, max 1784, avg 280
Masking 100%                                                                                                                                                             

In [32]:
# filter table to match filtered sequence file
! qiime feature-table filter-features \
    --i-table ./table-no-ecmu.qza \
    --m-metadata-file ./hits.qza \
    --o-filtered-table ./table-no-ecmu-hits.qza

Saved FeatureTable[Frequency] to: ./table-no-ecmu-hits.qza


#### Given that we filtered our data again, you may want to re-generate the taxonomy plots. Use the prior taxonomy visualization commands above as a guid and run them below, with the new table:

In [33]:
# updated taxonomy barplot
! qiime taxa barplot \
    --i-table ./table-no-ecmu-hits.qza \
    --i-taxonomy ./taxonomy.qza \
    --m-metadata-file ./metadata.tsv \
    --o-visualization ./table-no-ecmu-hits-taxa-barplot.qzv

Saved Visualization to: ./table-no-ecmu-hits-taxa-barplot.qzv


In [34]:
# updated krona plot
! qiime krona collapse-and-plot \
    --i-table ./table-no-ecmu-hits.qza \
    --i-taxonomy ./taxonomy.qza \
    --o-krona-plot ./table-no-ecmu-hits-taxa-krona.qzv

Saved Visualization to: ./table-no-ecmu-hits-taxa-krona.qzv


In [39]:
! qiime feature-table group \
    --i-table ./table-no-ecmu-hits.qza \
    --m-metadata-file ./metadata.tsv \
    --m-metadata-column 'genotype' \
    --p-mode 'mean-ceiling' \
    --p-axis 'sample'\
    --o-grouped-table ./table-no-ecmu-hits-genotype.qza

Saved FeatureTable[Frequency] to: ./table-no-ecmu-hits-genotype.qza


#### krona collapse by group 

In [13]:
! qiime feature-table group \
    --i-table ./table-no-ecmu-hits.qza \
    --p-axis sample \
    --m-metadata-file ./metadata.tsv \
    --m-metadata-column donor \
    --p-mode 'mean-ceiling' \
    --o-grouped-table ./table-no-ecmu-hits-donor.qza

Saved FeatureTable[Frequency] to: ./table-no-ecmu-hits-donor.qza


In [14]:
! qiime krona collapse-and-plot \
    --i-table ./table-no-ecmu-hits-donor.qza \
    --i-taxonomy ./taxonomy.qza \
    --o-krona-plot ./table-no-ecmu-hits-donor-taxa-krona.qzv

Saved Visualization to: ./table-no-ecmu-hits-donor-taxa-krona.qzv


## Construct phylogeny

See the [Inferring Phylogenies tutorial](https://docs.qiime2.org/2022.11/tutorials/phylogeny/) for more information.

We'll run [FastTree](https://docs.qiime2.org/2022.11/tutorials/phylogeny/#fasttree) to be quick, though I'd recomend [iqtree](https://docs.qiime2.org/2022.11/tutorials/phylogeny/#iqtree) or [fragment-insertion](https://library.qiime2.org/plugins/q2-fragment-insertion/16/).

We'll be using the [align-to-tree-mafft-fasttree](https://docs.qiime2.org/2022.11/tutorials/phylogeny/#pipelines) pipeline.

### *de novo phylogeny*

View with [iTOL](https://itol.embl.de/) or [Empress](https://github.com/biocore/empress).

In [16]:
# pipeline: alignment through phylogeny
! qiime phylogeny align-to-tree-mafft-fasttree \
    --i-sequences ./hits.qza \
    --output-dir ./mafft-fasttree-output \
    --verbose

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: mafft --preservecase --inputorder --thread 1 /tmp/qiime2/fmshaik/data/c0d7e485-8e8f-4018-9865-3ee92164fdae/data/dna-sequences.fasta

inputfile = orig
263 x 150 - 150 d
nthread = 1
nthreadpair = 1
nthreadtb = 1
ppenalty_ex = 0
stacksize: 8192 kb
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..
  201 / 263 (thread    0)
done.

Constructing a UPGMA tree (efffree=0) ... 
  260 / 263
done.

Progressive alignment 1/2... 
STEP   148 / 262 (thread    0)
Reallocating..done. *alloclen = 1301
STEP   262 / 262 (thread    0) h
done.

Making a distance matrix from msa.. 
  200 / 263 (thread    0)
done.

Constructing a UPGMA tree (efffree=1) ... 
  260 / 263
done.

Progressive alignment 2/2... 


### Another phylogenetic approach: Fragment Insertion

In [16]:
sepp_ref='/home/SE/BMIG-6202-MSR/RefDBs/sepp-refs-silva-128.qza'

In [17]:
! qiime fragment-insertion sepp \
    --i-representative-sequences ./hits.qza \
    --i-reference-database $sepp_ref \
    --o-tree ./tree.qza \
    --o-placements ./tree_placements.qza \
    --p-threads 8

^C

Aborted!


In [ ]:
!  qiime fragment-insertion filter-features \
    --i-table ./table-no-ecmu-hits.qza \
    --i-tree ./tree.qza \
    --o-filtered-table ./table-no-ecmu-fi.qza \
    --o-removed-table ./table-no-ecmu-nofi.qza

In [ ]:
! qiime feature-table filter-seqs \
    --i-data ./hits.qza \
    --i-table ./table-no-ecmu-fi.qza \
    --o-filtered-data repset-no-ecmu-fi.qza

## [Empress](https://github.com/biocore/empress)

In [18]:
!qiime empress tree-plot \
    --i-tree ./mafft-fasttree-output/rooted_tree.qza \
    --m-feature-metadata-file ./taxonomy.qza \
    --o-visualization ./tree-viz.qzv

Saved Visualization to: ./tree-viz.qzv


In [ ]:
q2.Visualization.load('./tree-viz.qzv')

In [19]:
! qiime empress community-plot \
    --i-tree ./mafft-fasttree-output/rooted_tree.qza \
    --i-feature-table ./table-no-ecmu-hits.qza \
    --m-sample-metadata-file ./metadata.tsv \
    --m-feature-metadata-file ./taxonomy.qza \
    --o-visualization tree-tax-table.qzv

Saved Visualization to: tree-tax-table.qzv


In [ ]:
q2.Visualization.load('./tree-tax-table.qzv')